# Projet Getaround
# Exploration, nettoyage et préparation des données
# Production de restitutions d'analyses des impacts de mise en oeuvre de délai entre 2 locations

## Import des modules

In [168]:
import numpy as np
import pandas as pd
import plotly.express as px


## Chargement des données

In [ ]:
df = pd.read_excel("./data/get_around_delay_analysis.xlsx", sheet_name="rentals_data")

display(df.sample(10))
display(df.shape)
display(df.columns)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
14892,539499,229302,mobile,canceled,NaN,NaN,NaN
1617,538610,385721,connect,ended,31.0,NaN,NaN
5046,553236,389785,connect,ended,222.0,NaN,NaN
6344,544030,404771,mobile,ended,72.0,543848.0,120.0
1191,544634,401077,mobile,ended,124.0,NaN,NaN
21066,562438,255326,mobile,ended,-127.0,NaN,NaN
7683,513740,338940,mobile,ended,83.0,565837.0,0.0
10752,545073,271582,mobile,ended,7.0,NaN,NaN
11699,556325,396009,connect,ended,-50.0,NaN,NaN
15304,549186,383462,connect,ended,-87.0,NaN,NaN


(21310, 7)

Index(['rental_id', 'car_id', 'checkin_type', 'state',
       'delay_at_checkout_in_minutes', 'previous_ended_rental_id',
       'time_delta_with_previous_rental_in_minutes'],
      dtype='object')

rental_id	                                Identifiant unique de la location

car_id                        	            Identifiant unique de la voiture

checkin_type	                            Flux utilisé à la fois pour l’enregistrement et le départ.
                                            (c’est-à-dire l’accès et la restitution de la voiture)
                                            mobile = contrat de location signé sur le smartphone du propriétaire
                                            connect = voiture équipée de la technologie Connect, ouverte par le conducteur avec son smartphone.
                                            Remarque : les contrats papier ont été exclus des données car nous n’avons pas de données sur leur retard à la caisse et son cas d’utilisation est négligeable

state	                                    Annulée signifie que la location n’a pas eu lieu (a été annulée par le conducteur ou le propriétaire).

delay_at_checkout_in_minutes	            Différence en minutes entre l’heure de fin de location demandée par le chauffeur lors de la réservation de la voiture et l’heure réelle à laquelle le chauffeur a terminé le départ.
                                            Les valeurs négatives signifient que le conducteur a rendu la voiture à l’avance.

previous_ended_rental_id	                id de la location précédente de la voiture terminée (NULL en l’absence de location précédente ou retard avec la location précédente supérieur à 12 heures)

time_delta_with_previous_rental_in_minutes	Différence en minutes entre l’heure de début prévue de la location et l’heure de fin prévue de la location précédente (si inférieure à 12 heures, NULL si supérieure)

## Nettoyage et mise en forme des données

In [170]:
# Transformation en "str" des id
df['rental_id'] = df['rental_id'].astype(str)
df['car_id'] = df['car_id'].astype(str)
df['previous_ended_rental_id'] = df['previous_ended_rental_id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)


# Ajour de colonne au df
# colonne Retard constaté sur la location précedente (quand cette location existe)
df["previous_delay_at_checkout_in_minutes"] = df["previous_ended_rental_id"].apply(lambda id : id if pd.isna(id) else df[df["rental_id"] == id]["delay_at_checkout_in_minutes"].sum())

## Exploration des données

In [171]:
df.describe(include="all")

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_delay_at_checkout_in_minutes
count,21310,21310,21310,21310,16346.000000,1841,1841.000000,1841.000000
unique,21310,8143,2,2,NaN,1788,NaN,NaN
top,505000,334190,mobile,ended,NaN,537243,NaN,NaN
freq,1,33,17003,18045,NaN,4,NaN,NaN
mean,NaN,NaN,NaN,NaN,59.701517,NaN,279.288430,-23.255296
std,NaN,NaN,NaN,NaN,1002.561635,NaN,254.594486,417.333377
min,NaN,NaN,NaN,NaN,-22433.000000,NaN,0.000000,-4624.000000
25%,NaN,NaN,NaN,NaN,-36.000000,NaN,60.000000,-49.000000
50%,NaN,NaN,NaN,NaN,9.000000,NaN,180.000000,0.000000
75%,NaN,NaN,NaN,NaN,67.000000,NaN,540.000000,39.000000


In [172]:
df_cnt = df["checkin_type"].value_counts()
display(df_cnt)
fig = px.pie(df_cnt, values = "count", names=df_cnt.index)
fig.show()

checkin_type
mobile     17003
connect     4307
Name: count, dtype: int64

In [173]:
df_cnt = df["state"].value_counts()
display(df_cnt)
fig = px.pie(df_cnt, values = "count", names=df_cnt.index)
fig.show()

state
ended       18045
canceled     3265
Name: count, dtype: int64

In [174]:
# Distribution des diff. entre fin de locations et location suivante

df_cnt = df["time_delta_with_previous_rental_in_minutes"].value_counts()
display(df_cnt)
fig = px.bar(df_cnt, y = "count", x=df_cnt.index, labels=df_cnt.index)
fig.show()

time_delta_with_previous_rental_in_minutes
0.0      279
60.0     183
120.0    137
720.0    130
30.0     122
180.0     83
90.0      82
660.0     72
600.0     72
150.0     67
240.0     67
690.0     65
630.0     53
210.0     48
570.0     44
480.0     42
420.0     41
360.0     41
270.0     38
300.0     38
540.0     37
510.0     36
330.0     26
450.0     23
390.0     15
Name: count, dtype: int64

In [175]:
# Distribution hors :
#           Avances
#           Retards > 12h

df_disp = df[(df["delay_at_checkout_in_minutes"] > 0) & (df["delay_at_checkout_in_minutes"] <= 720)]
fig = px.histogram(df_disp, x="delay_at_checkout_in_minutes")
fig.show()

## Analyse d'impacts en fonction de seuil retenu


In [176]:
# création de dataframe : impacts en fct de seuil
# Seuils de 0 à 12h par tranche de 30mn
df_impacts_seuil = pd.DataFrame()

df_impacts_seuil["seuil"] = [s for s in range(0, 6*60+1, 30)]

list_locs_impactee = []
list_retards = [] 

for seuil in df_impacts_seuil["seuil"] :

    nb_locs_impactee = ((df["time_delta_with_previous_rental_in_minutes"] < seuil)).sum()
    list_locs_impactee.append(nb_locs_impactee)
    
    nb_retards = ((df["previous_delay_at_checkout_in_minutes"] > seuil)).sum()
    list_retards.append(nb_retards)


df_impacts_seuil["locs_impactee"] = list_locs_impactee
df_impacts_seuil["locs_retards"] = list_retards

In [177]:
df_impacts_seuil

,seuil,locs_impactee,locs_retards
0,0,0,873
1,30,279,509
2,60,401,339
3,90,584,216
4,120,666,156
5,150,803,115
6,180,870,91
7,210,953,76
8,240,1001,71
9,270,1068,52


In [ ]:
# En fct de chaque seuil, potentiellement : le nombre de locations impactées et les retards couverts  

fig = px.line(df_impacts_seuil, y = ["locs_impactee", "locs_retards"], x="seuil", labels=df_cnt.index, title="cheking_type = tous")
fig.show()

## Analyse d'impacts en fonction de seuil retenu (canal = connect)

In [182]:
# création de dataframe : impacts en fct de seuil
# Seuils de 0 à 12h par tranche de 30mn
df_impacts_seuil = pd.DataFrame()

df_impacts_seuil["seuil"] = [s for s in range(0, 6*60+1, 30)]

list_locs_impactee = []
list_retards = [] 

for seuil in df_impacts_seuil["seuil"] :

    nb_locs_impactee = ((df["checkin_type"] == "connect") & (df["time_delta_with_previous_rental_in_minutes"] < seuil)).sum()
    list_locs_impactee.append(nb_locs_impactee)
    
    nb_retards = ((df["checkin_type"] == "connect") & (df["previous_delay_at_checkout_in_minutes"] > seuil)).sum()
    list_retards.append(nb_retards)


df_impacts_seuil["locs_impactee"] = list_locs_impactee
df_impacts_seuil["locs_retards"] = list_retards

fig = px.line(df_impacts_seuil, y = ["locs_impactee", "locs_retards"], x="seuil", labels=df_cnt.index, title="cheking_type = connect")
fig.show()

In [181]:
# création de dataframe : impacts en fct de seuil
# Seuils de 0 à 12h par tranche de 30mn
df_impacts_seuil = pd.DataFrame()

df_impacts_seuil["seuil"] = [s for s in range(0, 6*60+1, 30)]

list_locs_impactee = []
list_retards = [] 

for seuil in df_impacts_seuil["seuil"] :

    nb_locs_impactee = ((df["checkin_type"] == "mobile") & (df["time_delta_with_previous_rental_in_minutes"] < seuil)).sum()
    list_locs_impactee.append(nb_locs_impactee)
    
    nb_retards = ((df["checkin_type"] == "mobile") & (df["previous_delay_at_checkout_in_minutes"] > seuil)).sum()
    list_retards.append(nb_retards)


df_impacts_seuil["locs_impactee"] = list_locs_impactee
df_impacts_seuil["locs_retards"] = list_retards

fig = px.line(df_impacts_seuil, y = ["locs_impactee", "locs_retards"], x="seuil", labels=df_cnt.index, title="cheking_type = mobile")
fig.show()

In [3]:
import plotly as px
print(px.__version__)

5.24.1
